In [1]:
import sys
sys.path.append('../')
import daq
import file
import globalVAR as Gvar
import visa
import numpy as np
import time
from random import randint
import PyCapture2 as pc2

%load_ext autoreload
%autoreload 1

In [2]:
%aimport daq
DAQ = daq.Daq()
DAQ.connect_instr('TDS2024C', 1)
DAQ.connect_instr('KA3005P', 'ttyUSB0')
DAQ.connect_instr('Camera', 17583372)
DAQ.connect_instr('Camera', 17529184)
#DAQ.connect_instr('HR4000', 'michaelito')

Oscilloscope ID: TEKTRONIX,TDS 2024C,C046401,CF:91.1CT FV:v24.26

Power supply ID: KORADKA3005PV2.0
Camera ID: Blackfly BFLY-PGE-31S4M,17583372,FV:1.61.3.0
Camera ID: Blackfly BFLY-PGE-50S5M,17529184,FV:1.53.3.0


In [3]:
start1 = daq.Command(DAQ.instr["17583372"].start_capture, response=False)
start2 = daq.Command(DAQ.instr["17529184"].start_capture, response=False)
pic1 = daq.Command(DAQ.instr["17583372"].retrieve_buffer, response=True)
pic2 = daq.Command(DAQ.instr["17529184"].retrieve_buffer, response=True)
stop1 = daq.Command(DAQ.instr["17583372"].stop_capture, response=False)
stop2 = daq.Command(DAQ.instr["17529184"].stop_capture, response=False)

In [4]:
serial = "None"
v = np.linspace(0, 20, 1000)
for i in range(1000):
    com = daq.Command(DAQ.instr[serial].set_voltage, (v[i],), False)
    DAQ.command_queue[serial].put(com)

In [5]:
dataSet = Gvar.getDataSetNum()
file.add_to_log(dataSet)
file.make_dir_struct('META', dataSet)
file.make_dir_struct('IMAGE', dataSet)
def command1(shot):
    print('Take %d at %0.3f' % (shot, time.clock()))
    return DAQ.instr["17583372"].retrieve_buffer()

def callback(ret, shot, serial):
    #print('Save %d at %0.3f - b' % (shot, time.clock()))
    data = {}
    data['meta'] = {'Serial number' : serial}
    data['image'] = ret
    #file.save_IMAGE(data, dataSet, shot)
    #print('Save %d at %0.3f - end' % (shot, time.clock()))

DAQ.command_queue["17583372"].put(start1)
for i in range(10):
    com = daq.Command(command1, callback=callback, args=(i,), callargs=(i,17583372))
    DAQ.command_queue["17583372"].put(com)
DAQ.command_queue["17583372"].put(stop1)

Take 0 at 0.870
Take 1 at 0.921
Take 2 at 0.958
Take 3 at 0.995
Take 4 at 1.031
Take 5 at 1.067
Take 6 at 1.104
Take 7 at 1.140
Take 8 at 1.176
Take 9 at 1.213


In [9]:
dataSet = Gvar.getDataSetNum()
file.add_to_log(dataSet)
file.make_dir_struct('META', dataSet)
file.make_dir_struct('IMAGE', dataSet)
com = daq.Command(DAQ.instr["17583372"].set_frame_rate, response=False, args=(10,))
DAQ.command_queue["17583372"].put(com)
def sav_callback(image, shot, serial):
    print('Callback')
    #print('Save %d at %0.3f - b' % (shot, time.clock()))
    #data = {}
    #data['meta'] = {'Serial number' : serial}
    #data['image'] = ret
    #name = file.save_IMAGE(data, 1807090040, randint(0, 100))
    #print(image)
    #image.save(bytes('test1.tiff', 'utf-8'), pc2.IMAGE_FILE_FORMAT.TIFF)
    #print('Save %d at %0.3f - end' % (shot, time.clock()))

def cam_callback(image, args):
    response = daq.Response(image, sav_callback, args)
    #image.save(bytes('test1.tiff', 'utf-8'), pc2.IMAGE_FILE_FORMAT.TIFF)
    DAQ.response_queue["17583372"].put(response)
    print(time.clock())

def command(shot, serial):
    DAQ.instr["17583372"].start_capture(cam_callback, (shot, serial))

com = daq.Command(command, response=False, args=(0, "17583372"), wait=1)
DAQ.command_queue["17583372"].put(com)
DAQ.command_queue["17583372"].put(stop1)

3.835603
3.877146
3.917672
3.98333
4.013953
4.0437
4.079419
4.112563
4.146068
4.186884
4.24954


In [9]:
names = []
for name in DAQ.instr:
    names.append(name)
for name in names:
    DAQ.disconnect_instr(name)

Take 98 at 9.165
Take 99 at 9.193
